In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os, codecs, string, random
from numpy.random import seed as random_seed
from numpy.random import shuffle as random_shuffle

%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings; warnings.simplefilter('ignore')

import spacy, nltk, gensim
import pyLDAvis.gensim

import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

In [4]:
seed = 1
random.seed(seed)

In [5]:
dirnames = "data/"+os.listdir("data")[0]
dirnames += "/"+ os.listdir(dirnames)[0]+"/"

### EDA
let's get a look at all the different types of documents we will be dealing with

In [6]:
from tqdm import tqdm


In [7]:
os.listdir(dirnames)

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [8]:
# data = pd.DataFrame()
# for dirname in tqdm(os.listdir(dirnames)):
#     print(dirname)
#     for file in os.listdir(dirnames + dirname):
#         doc = open(file, 'r', errors='ignore').read()
#         data.append(doc)

In [9]:
# gather some information: 
#      if not empty: newsgroups, subject, organization, date, lines
# split on :"Lines: "

# separate file into "lines" with separator ">" si useless
# remove any line containing @ : split on "/n" and remove if in
# remove stopwords
# remove any empty "line"
# lowercase but watch out for names
# for each folder / category: separate word counts
# merge all into global word count while keeping the others


In [10]:
import re

In [188]:
# should make it into a reading function of course!

In [211]:
atheism = pd.DataFrame(columns = ["id","newsgroups","subject","organization","date","n_lines","text"])
dirname = dirnames + os.listdir(dirnames)[0] + "/"
for filename in os.listdir(dirname):
    file = dirname + filename
    with open(file) as current_file:
#         should check that all files have the same number of digits in the IID :////
        id = current_file.name[-5:]
        doc = current_file.read()
        if(re.search("Newsgroups: .*\n", doc) and re.findall("Newsgroups: .*\n", doc)[0]):
            newsgroups = re.findall("Newsgroups: .*\n", doc)[0]\
                            .split("\n")[0].split("Newsgroups: ")[-1].split(",")
            
        if(re.search("Subject: .*\n", doc) and re.findall("Subject: .*\n", doc)[0]):
            subject = re.findall("Subject: .*\n", doc)[0]\
                            .split("\n")[0].split("Subject: ")[-1]
            
        if(re.search("Organization: .*\n", doc) and re.findall("Organization: .*\n", doc)[0]):
            organization = re.findall("Organization: .*\n", doc)[0]\
                            .split("\n")[0].split("Organization: ")[-1]
            
        if(re.search("Date: .*\n", doc) and re.findall("Date: .*\n", doc)[0]):
            date = re.findall("Date: .*\n", doc)[0]\
                            .split("\n")[0].split("Date: ")[-1]
            
        if(re.search("Lines: .*\n", doc) and re.findall("Lines: .*\n", doc)[0]):
            n_lines = re.findall("Lines: .*\n", doc)[0]\
                            .split("\n")[0].split("Lines: ")[-1]
            
    #         make sure it exists
#         text = "".join(re.split(".*@.*","".join(re.split("\nLines: [0123456789]+\n\n", doc)[1:])))
# were saying that there always exists these stuff,,, basically we'll drop anything not having these
# since we're taking the first split to be the preamble, if there is no first preamble split but one later, it will
# remove part of the text, but we will say that this doesnt happen too often
#  also basically we're saying that "Lines" is a stopword! since we're splitting on it :/ 
# we could split and add it back but it makes sense to keep it as a stopword in most cases except geometry / else.... 
# but at the same time, looking at the list of stopwords we are going to use later, there are words such as 
# "keep" which has th same issue! so we're fine.
        text = "".join(re.split("\nLines: [0123456789]+\n", doc)[1:])
        atheism = atheism.append(pd.DataFrame(np.array([[id, newsgroups, subject, organization, date, n_lines, text]]),\
                                    columns=["id","newsgroups","subject","organization","date","n_lines","text"]))

In [148]:
data_backup = atheism.copy(deep = True)

In [133]:
atheism.head(10)

,id,newsgroups,subject,organization,date,n_lines,text
0,49960,"[alt.atheism, alt.atheism.moderated, news.answ...",Alt.Atheism FAQ: Atheist Resources,"Mantis Consultants, Cambridge. UK.","Mon, 29 Mar 1993 11:57:19 GMT",290,\nArchive-name: atheism/resources\nAlt-atheism...
0,51060,"[alt.atheism, alt.atheism.moderated, news.answ...",Alt.Atheism FAQ: Introduction to Atheism,"Mantis Consultants, Cambridge. UK.","Mon, 5 Apr 1993 12:22:45 GMT",646,\nArchive-name: atheism/introduction\nAlt-athe...
0,51119,[alt.atheism],Re: Gospel Dating,"Technical University Braunschweig, Germany","Mon, 5 Apr 1993 19:08:25 GMT",93,\nIn article <65974@mimsy.umd.edu>\nmangoe@cs....
0,51120,"[alt.atheism, alt.politics.usa.constitution]",Re: university violating separation of church/...,"Mantis Consultants, Cambridge. UK.","Mon, 5 Apr 1993 17:58:42 GMT",29,\ndmn@kepler.unh.edu (...until kings become ph...
0,51121,"[alt.atheism, soc.motss, rec.scouting]","Re: [soc.motss, et al.] ""Princeton axes matchi...",IBM Research,"Mon, 05 Apr 93 18:01:16 GMT",15,\nIn article <N4HY.93Apr5120934@harder.ccr-p.i...
0,51122,[alt.atheism],Re: A visit from the Jehovah's Witnesses,"Technical University Braunschweig, Germany","Mon, 5 Apr 1993 19:24:19 GMT",114,\nIn article <1993Apr5.091139.823@batman.bmd.t...
0,51123,[alt.atheism],Re: Political Atheists?,"California Institute of Technology, Pasadena",2 Apr 93 19:05:57 GMT,11,NNTP-Posting-Host: punisher.caltech.edu\n\narr...
0,51124,[alt.atheism],Re: An Anecdote about Islam,"Technical University Braunschweig, Germany","Mon, 5 Apr 1993 19:45:34 GMT",28,\nIn article <114127@bu.edu>\njaeger@buphy.bu....
0,51125,[alt.atheism],Re: <Political Atheists?,"California Institute of Technology, Pasadena",2 Apr 1993 20:43:17 GMT,54,Message-ID: <1pi8h5INNq40@gap.caltech.edu>\nRe...
0,51126,[alt.atheism],Re: >>>>>>Pompous ass,"California Institute of Technology, Pasadena",2 Apr 93 20:57:33 GMT,9,NNTP-Posting-Host: punisher.caltech.edu\n\nkmr...


In [135]:
atheism.shape

(1000, 7)

In [136]:
nlp = spacy.load("en_core_web_sm")

In [137]:
from spacy.lang.en.stop_words import STOP_WORDS

In [138]:
# if needed
example = nlp(atheism.iloc[0,5])[0:100]
example

290

In [139]:
[(token.text, token.lemma_, token.pos_) for token in example][0:10]

[('290', '290', 'NUM')]

In [140]:
from collections import Counter
from string import punctuation

In [141]:
EXCLUDE_CHARS = list(set(punctuation).union(set('’')))

In [142]:
" ".join(EXCLUDE_CHARS)

'[ - ! . ’ ( | % + , * " ) ] & \' ; > ` < _ $ } ? @ = \\ { # : ~ / ^'

In [193]:
# remove stopwords and then lemmatize
def lemmatizer(doc):
    words = [token.lemma_ for token in nlp(doc) if token.is_stop != True \
             and token.is_punct != True \
             and token.is_space != True \
             and re.search("\\"+"|\\".join(EXCLUDE_CHARS), token.lemma_) is None]
    return words

In [174]:
# still getting some weird ones, so: remove ">", so should find a list of special characters to escape
# we re not doing name bundling and we're not taking care of proper nouns / etc, for another time
#     not lemmatized properly: still gettin some ve , s etc
#         not all lower cased
#         remove any word that contains no space but @ or "." in the middle of it
# DONE

In [212]:
%reset_selective -f temp
temp = atheism.copy(deep = True)
temp = temp.iloc[0:10]
# to test out stuff first

In [213]:
from tqdm.auto import tqdm  # for notebooks
from tqdm import tqdm, tqdm_pandas, tqdm_notebook

In [215]:
# do this on the full doc 
temp.text = temp.text.apply(lambda x: lemmatizer(x))



  0%|                                                                                         | 0/2442 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████| 2442/2442 [00:00<00:00, 90264.47it/s]

  0%|                                                                                         | 0/6805 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████| 6805/6805 [00:00<00:00, 117931.93it/s]

  0%|                                                                                          | 0/950 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 950/950 [00:00<00:00, 57261.36it/s]

  0%|                                                                                          | 0/319 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 319/319 [00:00<00:00, 43911.49it/s]

  0%|                         

In [159]:
# lemmatizer works ok,
# now need to check the no data text rows, not normal.
# but is a slow method!.... maybe should do this when reading? probably not.

In [163]:
# should remove tags!

In [216]:
#  do this on the full doc 

bag_of_words = Counter(temp.text.explode())

In [217]:
len(bag_of_words)

1798

In [218]:
bag_of_words.most_common(10)

[('atheist', 100),
 ('god', 92),
 ('not', 67),
 ('the', 58),
 ('atheism', 57),
 ('be', 52),
 ('religion', 50),
 ('religious', 47),
 ('believe', 41),
 ('belief', 39)]

In [219]:
# should add some characters to the stop words list: "not" "the" and some others.... should check the stopword list we are using:/

In [ ]:
# should also implement a tfidf 

In [ ]:
# could filter out using Luhn or at least remove hapaxeS? lets see

In [ ]:
could also train a ngram probabilistic parser? but no truth :/

# now on to the non processing part

In [220]:
# could do sentiment analysis?
# could predict the tag / category ?